In [1]:
from scipy.io import loadmat
from utils import *
data = loadmat('emnist-digits.mat')

In [2]:
import matplotlib.pyplot as plt
import numpy as np


In [3]:
dataset = data['dataset'][0, 0]
train = dataset[0][0, 0]  
test = dataset[1][0, 0]  
mapping = dataset[2]

train_images = train['images']   # Shape: (N, 28*28)
train_labels = train['labels']  # Shape: (N, 1)
train_writers = train['writers']

In [4]:
X = train_images.astype(np.float32) / 255.0  # Normalize to [0, 1]
y = train_labels.flatten().astype(np.int64)

# Wrap into a datalist with a single client
datalist = [(X, y)]

test_images = test['images'].astype(np.float32) / 255.0
test_labels = test['labels'].flatten().astype(np.int64)


In [5]:
from features_utils import *

In [6]:
n_clients = 30
datalist = make_femnist_datasets(X,y,train,n_clients)
# Hyperparameters
T = 5       # number of global rounds
K = 10      # number of client GD steps
gamma = 0.1 # learning rate
print(f"case with {n_clients} clients, with feature distribution shift")
model = fedavg(datalist, T, K, gamma)


case with 30 clients, with feature distribution shift
round :  1
round :  2
round :  3
round :  4
round :  5


In [7]:
# Evaluate
test_accuracy = evaluate(model, test_images, test_labels)
print(f"Test Accuracy with {n_clients} ): {test_accuracy * 100:.2f}%")

Test Accuracy with 30 ): 84.90%


In [8]:


n_clients = 30
datalist = make_femnist_datasets(X,y,train,n_clients)

# Hyperparameters
T = 5       # number of global rounds
K = 10      # number of client GD steps
gamma = 0.1 # learning rate

# 1) Create MADE data loaders
made_loaders = [
    DataLoader(
        TensorDataset(
            torch.tensor(X, dtype=torch.float32),
            torch.zeros(len(X), dtype=torch.float32)
        ),
        batch_size=64,
        shuffle=True
    )
    for X, _ in datalist
]

# 2) Train global MADE
global_made = train_global_made(
    made_loaders,
    dim=28*28,
    hid=100,
    rounds=T,
    local_epochs=1
)

# 3) Compute sample‐weights α for each client
sample_weights = []
for ld in made_loaders:
    # train local MADE and load its weights
    local_made = MADE(28*28, 100)
    local_state = train_local_made(local_made, ld, epochs=1)
    local_made.load_state_dict(local_state)

    # compute α for this client (Tensor of shape [n_samples])
    alpha = compute_sample_weights(global_made, local_made, ld)
    sample_weights.append(alpha)

# 4) Build weighted datasets by oversampling
weighted_datalist = []
for (Xi, yi), a in zip(datalist, sample_weights):
    # normalize and scale to counts, then convert to numpy ints
    counts = (a / a.sum() * len(yi)).cpu().numpy().astype(int)
    idxs = np.repeat(np.arange(len(yi)), counts)
    weighted_datalist.append((Xi[idxs], yi[idxs]))

# 5) Federated training on weighted data
print(f"case with {n_clients} clients, with feature distribution shift")
model = fedavg(weighted_datalist, T, K, gamma)


case with 30 clients, with feature distribution shift
round :  1
round :  2
round :  3
round :  4
round :  5


In [9]:

test_accuracy = evaluate(model, test_images, test_labels)
print(f"Test Accuracy with {n_clients} ): {test_accuracy * 100:.2f}%")

Test Accuracy with 30 ): 84.10%
